In [174]:
import base64
import secrets
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

In [175]:
def generate_random_bytes(bytes_length):
    random_bytes = secrets.token_bytes(bytes_length)
    return random_bytes

In [176]:
STRING_TO_PREPEND = b"comment1=cooking%20MCs;userdata="
STRING_TO_APPEND = b";comment2=%20like%20a%20pound%20of%20bacon"
BLOCK_LENGTH = 16
KEY = generate_random_bytes(BLOCK_LENGTH)
IV = generate_random_bytes(BLOCK_LENGTH)
STRING_ADMIN = b";admin=true;"

In [177]:
def filter(message):
    res = []
    for i in range(len(message)):
        if message[i:i+1] == b";":
            res.append(b"%3B")
        elif message[i:i+1] == b"=":
            res.append(b"%3D")
        else:
            res.append(message[i:i+1])
    return b"".join(res)

# encrypt with padding first
def encrypt_CBC(message, key=KEY, iv=IV, block_len=BLOCK_LENGTH):
    assert len(key) == block_len
    padded_message = pad(STRING_TO_PREPEND + message + STRING_TO_APPEND, block_len)
    n_blocks = len(padded_message) // block_len
    message_block = iv
    encrypted_message = b""
    cipher = AES.new(key, AES.MODE_ECB)
    for i in range(n_blocks):
        xored_message = bytes([c1^c2 for c1, c2 in zip(padded_message[i*block_len:(i+1)*block_len], message_block)])
        message_block = cipher.encrypt(xored_message)
        encrypted_message += message_block
    return encrypted_message

def decrypt_CBC(message, key=KEY, iv=IV, block_len=BLOCK_LENGTH):
    assert len(key) == block_len
    assert len(message) % block_len == 0
    n_blocks = len(message) // block_len
    dmessage_block = iv
    cipher = AES.new(key, AES.MODE_ECB)
    original_message = b""
    for i in range(n_blocks):
        message_block = message[i*block_len:(i+1)*block_len]
        xored_message_block = cipher.decrypt(message_block) 
        original_message_block = bytes([c1^c2 for c1, c2 in zip(xored_message_block, dmessage_block)])
        original_message += original_message_block
        dmessage_block = message_block
    return original_message

def find_common_block_num(message1, message2):
    start = 0
    num = 0
    while start * BLOCK_LENGTH < min(len(message1), len(message2)):
        if message1[start * BLOCK_LENGTH:(start+1) * BLOCK_LENGTH] == message2[start * BLOCK_LENGTH:(start+1) * BLOCK_LENGTH]:
             num += 1
             start += 1
        else:
            break
    return num
         
def find_input_start_location():
    n_common = find_common_block_num(encrypt_CBC(b""), encrypt_CBC(b"A"))
    for i in range(1, BLOCK_LENGTH+1):
        n_common_update = find_common_block_num(encrypt_CBC(b"A"*i), encrypt_CBC(b"A"*(i+1)))
        if n_common_update == n_common + 1:
            break
    return n_common * BLOCK_LENGTH + BLOCK_LENGTH - i     

def verify_admin(encrypted_message):
    decrypted_message = decrypt_CBC(encrypted_message)
    if STRING_ADMIN in decrypted_message:
        return True
    else:
        return False

In [178]:
STRING_ADMIN

b';admin=true;'

In [179]:
find_input_start_location()

32

In [180]:
message1 = encrypt_CBC(b"")[BLOCK_LENGTH:2*BLOCK_LENGTH]
message2 = b";admin=true;" + b"aaaa"
message = bytes([s1^s2 for (s1, s2) in zip(message1, message2)])
encrypted_message = encrypt_CBC(message)[2*BLOCK_LENGTH:3*BLOCK_LENGTH]

In [181]:
crafted_message = b"\x00" * 16 + encrypted_message
verify_admin(crafted_message) 

True